This is a slight update to the JSD calculation used in plotFingerprint, mostly to catch bugs. As input, this takes the `--outRawCounts` file from `plotFingerprint`.

If correct, these samples are kind of crappy.

The `sys.path` stuff is just due to how I'm running this locally.

In [3]:
import sys
del sys.path[7]

In [15]:
f = open("CD4_100000_fix.rawcounts.txt")

import numpy as np
mat = []
labels = None
for line in f:
    if line.startswith("'"):
        labels = line.strip().split("\t")
        labels = [x.strip("'") for x in labels]
        continue
    cols = line.strip().split("\t")
    cols = [int(x) for x in cols]
    mat.append(cols)

mat = np.array(mat)
mat = mat.T

print labels

['CD4_100000_fix_H3K27ac', 'CD4_100000_fix_H3K27me3', 'CD4_100000_fix_H3K4me3', 'CD4_100000_fix_Input']


In [91]:
from scipy import interpolate
def getJSD(labels, reference, idx, mat):
    """
    Computes the Jensen-Shannon distance between two samples. This is essentially
    a symmetric version of Kullback-Leibler divergence. The implementation
    presented here is based on code from Sitanshu Gakkhar at BCGSC.
    Note that the interpolation has the effect of removing zero count coverage
    bins, which ends up being needed for the JSD calculation.
    args: The input arguments
    idx:  The column index of the current sample
    mat:  The matrix of counts
    """

    # Get the index of the reference sample
    if reference not in labels:
        return "NA"
    refIdx = labels.index(reference)
    if refIdx == idx:
        return "NA"

    # These will hold the coverage histograms
    chip = np.zeros(20000, dtype=np.int)
    input = np.zeros(20000, dtype=np.int)
    for row in mat:
        # ChIP
        val = row[idx]
        # N.B., we need to clip past the end of the array
        if val >= 20000:
            val = 19999
        # This effectively removes differences due to coverage percentages
        if val > 0:
            chip[int(val)] += 1

        # Input
        val = row[refIdx]
        if val >= 20000:
            val = 19999
        if val > 0:
            input[int(val)] += 1

    def signalAndBinDist(x):
        x = np.array(x)
        (n,) = x.shape
        signalValues = np.array(list(range(n)))
        totalSignal = x * signalValues
        normalizedTotalSignal = np.cumsum(totalSignal) / np.sum(totalSignal).astype("float")
        binDist = np.cumsum(x).astype("float") / sum(x)
        interpolater = interpolate.interp1d(binDist, normalizedTotalSignal, fill_value="extrapolate")
        return (binDist, normalizedTotalSignal, interpolater)

    # Interpolate the signals to evenly spaced bins, which also removes 0-coverage bins
    chipSignal = signalAndBinDist(chip)
    inputSignal = signalAndBinDist(input)

    # These are basically CDFs
    inputSignalInterp = inputSignal[2](np.arange(0, 1.00001, 0.00001))
    chipSignalInterp = chipSignal[2](np.arange(0, 1.00001, 0.00001))

    # If there are no low coverage bins then you can get nan as the first interpolated value.
    # That should instead be some small value
    if np.isnan(inputSignalInterp[0]):
        inputSignalInterp[0] = 1e-12
    if np.isnan(chipSignalInterp[0]):
        chipSignalInterp[0] = 1e-12

    # Differentiate to PMFs, do some sanity checking
    PMFinput = np.ediff1d(inputSignalInterp)
    PMFchip = np.ediff1d(chipSignalInterp)
    if abs(sum(PMFinput) - 1) > 0.01 or abs(sum(PMFchip) - 1) > 0.01:
        sys.stderr.write("Warning: At least one PMF integral is significantly different from 1! The JSD will not be returned")
        return "NA"

    # Compute the JSD from the PMFs
    M = (PMFinput + PMFchip) / 2.0
    JSD = 0.5 * (np.sum(PMFinput * np.log2(PMFinput / M))) + 0.5 * (np.sum(PMFchip * np.log2(PMFchip / M)))
    np.sqrt(JSD)

    return JSD

In [94]:
for i in range(len(labels)):
    foo = getJSD(labels, labels[-1], i, mat)
    print("{}: {}".format(labels[i], foo))

CD4_100000_fix_H3K27ac: 0.00592408514627
CD4_100000_fix_H3K27me3: 0.00360687521836
CD4_100000_fix_H3K4me3: 0.00684560175001
CD4_100000_fix_Input: NA
